In [ ]:
import numpy as np
import pandas as pd

READING DATA

In [ ]:
events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
events.head()

In [ ]:
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
oil

In [ ]:
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
stores.head()

In [ ]:
trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
trans.head()

In [ ]:
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
test.head()

MERGING DATA

In [ ]:
print(train.shape)

train_df = None

train_df = train.merge(stores, how='left', left_on='store_nbr', right_on = 'store_nbr')
train_df = train_df.merge(events, how='left', left_on='date', right_on = 'date')
train_df = train_df.merge(trans, how='left', left_on=['date', 'store_nbr'], right_on = ['date', 'store_nbr'])
train_df = train_df.merge(oil, how='left', left_on='date', right_on = 'date')
print(train_df.shape)

In [ ]:
print(test.shape)
test_df = None

test_df = test.merge(stores, how='left', left_on='store_nbr', right_on = 'store_nbr')
test_df = test_df.merge(events, how='left', left_on='date', right_on = 'date')
test_df = test_df.merge(trans, how='left', left_on=['date', 'store_nbr'], right_on = ['date', 'store_nbr'])
test_df = test_df.merge(oil, how='left', left_on='date', right_on = 'date')
print(test_df.shape)

In [ ]:
train_df.isna().sum()

DATA CLEANING

In [ ]:
train_df['type_y'] = train_df['type_y'].fillna('Day')
train_df['locale'] = train_df['locale'].fillna('National')
train_df['locale_name'] = train_df['locale_name'].fillna('None')
train_df['transferred'] = train_df['transferred'].fillna('False')
train_df['transactions'] = train_df['transactions'].fillna(0)
train_df['dcoilwtico'] = train_df['dcoilwtico'].fillna(0)
train_df = train_df.drop(columns=['id', 'description', 'locale_name'], axis = 1)

In [ ]:
train_df.isna().sum()

In [ ]:
test_df['type_y'] = test_df['type_y'].fillna('Day')
test_df['locale'] = test_df['locale'].fillna('National')
test_df['locale_name'] = test_df['locale_name'].fillna('None')
test_df['transferred'] = test_df['transferred'].fillna('False')
test_df['transactions'] = test_df['transactions'].fillna(0)
test_df['dcoilwtico'] = test_df['dcoilwtico'].fillna(0)

sub=pd.DataFrame()
sub['id'] = test_df['id']
test_df = test_df.drop(columns=['id', 'description', 'locale_name'], axis = 1)

In [ ]:
test_df.isna().sum()

FEATURE ENGINEERING

In [ ]:
train_df['holiday'] = train_df['locale'] + '_' + train_df['type_y']
train_df['location'] = train_df['city'] +',' + train_df['state']

In [ ]:
test_df['location'] = test_df['city'] +',' + test_df['state']
test_df['holiday'] = test_df['locale'] +',' + test_df['type_y']

In [ ]:
train_df = train_df.drop(columns = ['city', 'state', 'locale', 'type_y'], axis = 1)
test_df = test_df.drop(columns = ['city', 'state', 'locale', 'type_y'], axis = 1)

In [ ]:
train_df['family'].unique()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15,7))
train_df.groupby("family").sum()['sales'].plot(kind='bar')

In [ ]:
# 15
# Tools -Automotive + Hardware + Home Appliances  + Lawn and Garden +Players and Electronics
# Clothing - Beauty + Lingerie + Ladies Wear
# Lifestyle - Babycare + Books + Magazines + Celebration + School and Office Supplies
# Other Foods - Grocery2 + Pet Foods + Sea Food
# rest remains the same

In [ ]:
## Merged many categories from the family feature to observe change in the case.

In [ ]:
train_df['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)

train_df['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)

train_df['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)

train_df['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)

train_df['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

In [ ]:
test_df['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)

test_df['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)

test_df['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)

test_df['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)

test_df['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

In [ ]:
train_df['family'].unique()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15,7))
train_df.groupby("family").sum()['sales'].plot(kind='bar')

In [ ]:
train_df.sort_values(by=['store_nbr', 'date', 'family'], inplace=True)
test_df.sort_values(by=['store_nbr', 'date', 'family'], inplace=True)

In [ ]:
train_df[["year", "month", "day"]] = train_df["date"].str.split("-", expand = True)
train_df = train_df.drop(columns=['date'], axis = 1)

test_df[["year", "month", "day"]] = test_df["date"].str.split("-", expand = True)
test_df = test_df.drop(columns=['date'], axis = 1)

In [ ]:
train_df['onpromo_lag1'] = train_df['onpromotion'].shift(periods=1)
test_df['onpromo_lag1'] = test_df['onpromotion'].shift(periods=1)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
from datetime import datetime
train_df["day"] = train_df["day"].astype('category')
train_df["day"] = train_df["day"].cat.codes
train_df["month"] = train_df["month"].astype('category')
train_df["month"] = train_df["month"].cat.codes
train_df["year"] = train_df["year"].astype('category')
train_df["year"] = train_df["year"].cat.codes

train_df["type_x"] = train_df["type_x"].astype('category')
train_df["type_x"] = train_df["type_x"].cat.codes
train_df["holiday"] = train_df["holiday"].astype('category')
train_df["holiday"] = train_df["holiday"].cat.codes
train_df["family"] = train_df["family"].astype('category')
train_df["family"] = train_df["family"].cat.codes
train_df["location"] = train_df["location"].astype('category')
train_df["location"] = train_df["location"].cat.codes

train_df["transferred"] = train_df["transferred"].astype('category')
train_df["transferred"] = train_df["transferred"].cat.codes

In [ ]:
#train_df['sale_lag1'] = train_df['sale_lag1'].fillna(0) 
train_df['onpromo_lag1'] = train_df['onpromo_lag1'].fillna(0) 

In [ ]:
from datetime import datetime
test_df["day"] = test_df["day"].astype('category')
test_df["day"] = test_df["day"].cat.codes
test_df["month"] = test_df["month"].astype('category')
test_df["month"] = test_df["month"].cat.codes
test_df["year"] = test_df["year"].astype('category')
test_df["year"] = test_df["year"].cat.codes

test_df["type_x"] = test_df["type_x"].astype('category')
test_df["type_x"] = test_df["type_x"].cat.codes
test_df["holiday"] = test_df["holiday"].astype('category')
test_df["holiday"] = test_df["holiday"].cat.codes
test_df["family"] = test_df["family"].astype('category')
test_df["family"] = test_df["family"].cat.codes
test_df["location"] = test_df["location"].astype('category')
test_df["location"] = test_df["location"].cat.codes

test_df["transferred"] = test_df["transferred"].astype('category')
test_df["transferred"] = test_df["transferred"].cat.codes

In [ ]:
test_df['onpromo_lag1'] = test_df['onpromo_lag1'].fillna(0) 

MODEL

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df.loc[:, train_df.columns != 'sales'], train_df['sales'], test_size=0.33, random_state=42)

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor()
model.fit(train_df.loc[:, train_df.columns != 'sales'], train_df['sales'])
# score 3.8

In [ ]:
sub['sales'] = model.predict(test_df)

In [ ]:
sub.to_csv('submission123.csv', index=False)